# Notebook TFM Prototipo
<div style="background-color:#F2EDED;">
<br/>
<div>
<img src="https://uploads-ssl.webflow.com/614b1fe22fa8b90ef41aeffe/6265cb48f9496b1cefc9ab75_logotipo-mbit-39.png" width="200px" align="left" CLASS="TextWrap" style="background-color:#2a3f3f; margin-left: 10px;">
<img src="https://branding-guidelines.msf.es/esp/imgs/logo/Logo-01.jpg" width="100px" align="right" CLASS="TextWrap" style="background-color:#2a3f3f;">
</div>
<br/>
<br/>
<br/>
<div>
<h1><font color="#2a3f3f" size=4 style="margin-left: 10px;">MODELO DE PROBABILIDAD A TESTAR EN MÉDICOS SIN FRONTERAS</font></h1>
</div>
<br/>
<div style="text-align: right; margin-right: 10px; margin-bottom: 10px;">
<font color="#2a3f3f" size=3>Elio López Salamanca </font><br>
<font color="#2a3f3f" size=3>Sergio Israel Calleja Chimeno</font><br>
</div>
</div>

En este proyecto vamos a generar un caso de uso que ayude a MSF a detectar socios potenciales que puedan
dejar el testamento a favor de MSF.

En este notebook vamos a cargar el modelo, evaluar todo el dataset y generar una salida con 100 socios que el modelo ha dicho que es probable que tengan
programa herencias y otros 100 que no

Este notebook consiste en los siguientes pasos

1. **Lectura de datos**: lectura de los datos anteriormente preprocesados
2. **Carga de modelo**: carga del modelo entrenado
3. **Ejecucion modelo**: se ejecuta el modelo para todo los datos
4. **Generacion datos prototipo**: se generan los datos para el prototipo

## Imports e instalables necesarios

In [2]:
import pandas as pd
import numpy as np
import pickle

## Leemos los datos preprocesados

In [3]:
df_preprocessed = pd.read_parquet("msf_df_preprocessed.parquet")
df_preprocessed.shape

(687224, 97)

## Load Model

In [4]:
file = "classifier_final.pkl"
model_tfm = pickle.load(open(file, 'rb'))

### Evaluacion Todo Dataset

In [5]:
y_pred  = model_tfm.predict(df_preprocessed.drop(["cnt_id","cnt_programaherencias"], axis=1))

In [6]:
y_pred_proba  = model_tfm.predict_proba(df_preprocessed.drop(["cnt_id","cnt_programaherencias"], axis=1))

## Guardamos la salida para montar el prototipo

In [7]:
df_preprocessed["cnt_programaherencias_pred"] = y_pred
df_preprocessed["cnt_programaherencias_prob"] = y_pred_proba.max(axis=1)

In [8]:
## Generamos muestra para prototipo
df_api_msf = df_preprocessed.loc[(df_preprocessed["cnt_programaherencias"]==False) 
                                 & (df_preprocessed["cnt_programaherencias_pred"]==1)].sample(100)
df_api_msf_2 = df_preprocessed.loc[(df_preprocessed["cnt_programaherencias"]==False) 
                                 & (df_preprocessed["cnt_programaherencias_pred"]==0)].sample(100)

In [9]:
df_api_msf_union = pd.concat([df_api_msf, df_api_msf_2])

In [10]:
df_api_msf_union.to_csv("msf_api_sample.csv")